# Data Wrangling: Clean, Transform, Merge, Reshape

In [1]:
%pylab inline

import numpy as np
from numpy.random import randn

import pandas as pd
from pandas import Series, DataFrame

Populating the interactive namespace from numpy and matplotlib


## Combining and Merging Data Sets
### Database-style DataFrame Merges

In [2]:
df1 = DataFrame({'key': list('bbacaab'), 'data1': range(7)})
df2 = DataFrame({'key': list('abd'), 'data2': range(3)})

In [3]:
# 重叠列的列名当键
pd.merge(df1, df2)

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


In [5]:
df3 = DataFrame({'lkey': list('bbacaab'), 'data1': range(7)})
df4 = DataFrame({'rkey': list('abd'), 'data2': range(3)})

In [6]:
# 指定键
pd.merge(df3, df4, left_on='lkey', right_on='rkey')

,data1,lkey,data2,rkey
0,0,b,1,b
1,1,b,1,b
2,6,b,1,b
3,2,a,0,a
4,4,a,0,a
5,5,a,0,a


In [7]:
# outer 外连接求取键的并集
pd.merge(df3, df4, left_on='lkey', right_on='rkey', how='outer')

,data1,lkey,data2,rkey
0,0.0,b,1.0,b
1,1.0,b,1.0,b
2,6.0,b,1.0,b
3,2.0,a,0.0,a
4,4.0,a,0.0,a
5,5.0,a,0.0,a
6,3.0,c,NaN,NaN
7,NaN,NaN,2.0,d


In [9]:
# left join
pd.merge(df1, df2, how='left')

,data1,key,data2
0,0,b,1.0
1,1,b,1.0
2,2,a,0.0
3,3,c,NaN
4,4,a,0.0
5,5,a,0.0
6,6,b,1.0


In [10]:
# inner join
pd.merge(df1, df2, how='inner')

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


In [12]:
left = DataFrame({
    'key1': ['foo','foo','bar'],
    'key2': ['one','two','one'],
    'lval': [1,2,3]
})
right = DataFrame({
    'key1': ['foo','foo','bar','bar'],
    'key2': ['one','one','one','two'],
    'lval': [4,5,6,7]
})

# 根据多个键进行合并
pd.merge(left, right, on=['key1','key2'], how='outer')

,key1,key2,lval_x,lval_y
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


In [15]:
# 合并出现重复 column
pd.merge(left, right, on='key1')

,key1,key2_x,lval_x,key2_y,lval_y
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [16]:
# 自定义重复列名
pd.merge(left, right, on='key1', suffixes=('_left', '_right'))

,key1,key2_left,lval_left,key2_right,lval_right
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


merge 函数的参数

参数 | 说明
---|---
`left` | 参与合并的左侧 DataFrame
`right` | 参与合并的右侧 DataFrame
`how` | `inner`, `outer`, `left`, `right`，默认为 `inner` 
`on` | 用于连接的列名
`left_on` | 左侧 DataFrame 中用作连接键的 column
`right_on` | 右侧 DataFrame 中用作连接键的 column
`left_index` | 将左侧的行索引用作其连接键
`right_index` | 将右侧的行索引用作其连接键
`sort` | 根据连接键对合并后的数据进行排序，默认为 True
`suffixes` | 字符串值 tuple，用于追加重叠 column 的末尾 
`copy` | 设置为 False。某些特殊情况下避免将数据复制到结果数据结构中

### Merging on Index

In [17]:
left1 = DataFrame({'key': list('abaabc'), 'value': range(6)})
right1 = DataFrame({'group_val': [3.5, 7]}, index=['a','b'])

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [18]:
# 使用 left_index, right_index 说明索引当做连接键
pd.merge(left1, right1, left_on='key', right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [19]:
# 层次化数据
lefth = DataFrame({
    'key1': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'],
    'key2': [2000, 2001, 2002, 2001, 2002],
    'data': np.arange(5.)})
righth = DataFrame(
    np.arange(12).reshape((6, 2)),
    index=[['Nevada', 'Nevada', 'Ohio', 'Ohio', 'Ohio', 'Ohio'],
           [2001, 2000, 2000, 2000, 2001, 2002]],
    columns=['event1', 'event2'])

In [20]:
pd.merge(lefth, righth, left_on=['key1','key2'], right_index=True)

,data,key1,key2,event1,event2
0,0.0,Ohio,2000,4,5
0,0.0,Ohio,2000,6,7
1,1.0,Ohio,2001,8,9
2,2.0,Ohio,2002,10,11
3,3.0,Nevada,2001,0,1


In [21]:
pd.merge(lefth, righth, left_on=['key1','key2'], right_index=True, how='outer')

,data,key1,key2,event1,event2
0,0.0,Ohio,2000,4.0,5.0
0,0.0,Ohio,2000,6.0,7.0
1,1.0,Ohio,2001,8.0,9.0
2,2.0,Ohio,2002,10.0,11.0
3,3.0,Nevada,2001,0.0,1.0
4,4.0,Nevada,2002,NaN,NaN
4,NaN,Nevada,2000,2.0,3.0


In [22]:
# 合并索引
left2 = DataFrame([[1., 2.], [3., 4.], [5., 6.]], 
                  index=['a', 'c', 'e'],
                  columns=['Ohio', 'Nevada'])
right2 = DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                   index=['b', 'c', 'd', 'e'], 
                   columns=['Missouri', 'Alabama'])

In [23]:
pd.merge(left2, right2, how='outer', left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [24]:
# 使用 join 实例方法
left2.join(right2, how='outer')

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [25]:
# 指定键
left1.join(right1, on='key')

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


In [27]:
another = DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
                    index=['a', 'c', 'e', 'f'], 
                    columns=['New York', 'Oregon'])

# 向join传入一组DataFrame
left2.join([right2, another])

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0


### Concatenating Along an Axis

In [28]:
arr = np.arange(12).reshape((3,4))
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [29]:
# 沿着 index 轴连接 （预设）
np.concatenate([arr,arr])

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [30]:
# 沿着 column 轴连接
np.concatenate([arr,arr], axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In [31]:
s1 = Series([0,1], index=['a','b'])
s2 = Series([2,3,4], index=['c','d','e'])
s3 = Series([5,6], index=['f','g'])

# 沿着 index 轴连接
pd.concat([s1,s2,s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [32]:
# 沿着 column 轴连接，变成一个 DataFrame
pd.concat([s1,s2,s3], axis=1)

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [34]:
s4 = pd.concat([s1*5, s3])

# 取合并的交集
pd.concat([s1, s4], axis=1, join='inner')

,0,1
a,0,0
b,1,5


In [37]:
# 使用 key 参数在连接轴上创建一个层次化索引
result = pd.concat([s1,s2,s3], keys=['one','two','three'])
result

one    a    0
       b    1
two    c    2
       d    3
       e    4
three  f    5
       g    6
dtype: int64

In [38]:
result.unstack()

,a,b,c,d,e,f,g
one,0.0,1.0,NaN,NaN,NaN,NaN,NaN
two,NaN,NaN,2.0,3.0,4.0,NaN,NaN
three,NaN,NaN,NaN,NaN,NaN,5.0,6.0


In [39]:
# 沿着axis=1对Series合并，keys变成DataFrame的列头
pd.concat([s1,s2,s3], axis=1, keys=['one','two','three'])

,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [40]:
df1 = DataFrame(np.arange(6).reshape(3, 2), index=['a', 'b', 'c'],
                columns=['one', 'two'])
df2 = DataFrame(5 + np.arange(4).reshape(2, 2), index=['a', 'c'], 
                columns=['three', 'four'])

In [44]:
# 沿着axis=1对DataFrame合并
pd.concat([df1, df2], axis=1, keys=['group1','group2'])

group1     group2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [45]:
pd.concat({'group1': df1, 'group2': df2}, axis=1)

group1     group2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [46]:
# 管理层次化索引的创建方式
pd.concat([df1, df2], axis=1, keys=['group1','group2'],
         names=['upper', 'lower'])

upper group1     group2     
lower    one two  three four
a          0   1    5.0  6.0
b          2   3    NaN  NaN
c          4   5    7.0  8.0

In [48]:
df1 = DataFrame(np.random.randn(3, 4), columns=['a', 'b', 'c', 'd'])
df2 = DataFrame(np.random.randn(2, 3), columns=['b', 'd', 'a'])

# 不保留连接轴上的索引
pd.concat([df1, df2], ignore_index=True)

,a,b,c,d
0,0.670673,-1.040290,-0.663562,2.132797
1,-0.594446,0.947202,0.383379,-0.663607
2,0.929357,-0.847709,-0.142845,0.856133
3,-2.266376,-1.139167,NaN,-0.542311
4,0.077403,1.519307,NaN,0.165559


concat 函数的参数

参数 | 说明
---|---
`objs` | 参与连接的pandas对象的列表或字典
`axis` | 指明连接的轴向
`join` | `inner`, `outer` 其中之一
`join_axes` | 指明用于其他 n-1 条轴的索引，不执行并集、交集运算
`keys` | 与连接对象有关的值，用于形成连接轴向上的层次化索引
`levels` | 指定用作层次化索引各级上的索引
`names` | 用于创建分层级别的名称
`verify_integrity` | 检查结果对象新轴上的重复情况，如果发现则引发异常。默认 False
`ignore_index` | 不保留连接轴上的索引，产生一组新索引 range

### Combining Data with Overlap

In [51]:
a = Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan], index=['f', 'e', 'd', 'c', 'b', 'a'])
b = Series(np.arange(len(a), dtype=np.float64), index=['f', 'e', 'd', 'c', 'b', 'a'])
b[-1] = np.nan

In [53]:
# 选择性合并
np.where(pd.isnull(a), b, a)

array([ 0. ,  2.5,  2. ,  3.5,  4.5,  nan])

In [55]:
# Series.combine_first 合并，且进行数据对齐
b[:-2].combine_first(a[2:])

a    NaN
b    4.5
c    3.0
d    2.0
e    1.0
f    0.0
dtype: float64

In [56]:
df1 = DataFrame({'a': [1., np.nan, 5., np.nan], 
                 'b': [np.nan, 2., np.nan, 6.],
                 'c': range(2, 18, 4)})
df2 = DataFrame({'a': [5., 4., np.nan, 3., 7.], 
                 'b': [np.nan, 3., 4., 6., 8.]})

In [57]:
# 用参数对象中的数据为调用者对象的缺失数据“打补丁”
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


## Reshaping and Pivoting
### Reshaping with Hierarchical Indexing
- `stack`：将数据的 column “旋转”为 row
- `unstack`：将数据的 row “旋转”为 column

In [58]:
data = DataFrame(np.arange(6).reshape((2, 3)),
                 index=pd.Index(['Ohio', 'Colorado'], name='state'),
                 columns=pd.Index(['one', 'two', 'three'], name='number'))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [59]:
result = data.stack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

In [60]:
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5
